In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
from github import Github
from creds import GH_TOKEN
from retrying import retry
import matplotlib.pyplot as plt

%matplotlib inline

In [24]:
fh = open('../data/netdata_github_users.csv')
n = 0
data = []
while n<10 :
    line = fh.readline()
    if ',' in line:
        row = ','.join(line.split(',')[2:])
        data.append(row)
    if not line:
        break
    n += 1
fh.close()

In [30]:
row.replace('\n','')
#data 
#pd.DataFrame(data)

"[datetime.datetime(2015, 10, 21, 9, 18, 6), 'Alexander Knorr', '************', 'opexxx@gmail.com', 'Nuremberg', 140, 469, 'https://api.github.com/users/opexxx', 'https://de.linkedin.com/in/alexander-knorr-1981\\r\\n\\r\\nhttps://keybase.io/opexxx\\r\\n\\r\\nhttps://www.twitter.com/opexxx\\r\\n', 372, 99, 'http://about.me/opexxx', datetime.datetime(2009, 7, 20, 11, 49, 34), datetime.datetime(2019, 9, 13, 8, 50, 41)]"

In [59]:
@retry(stop_max_attempt_number=3)
def do_geocode(location,geolocator):
    ''' Function to use geopy to do a geocode, with retry as it fails sometimes
    '''
    res = geolocator.geocode(location,addressdetails=True)
    return res

In [69]:
import time 
time.time()

1568896331.0316303

In [31]:
g.rate_limiting_resettime

NameError: name 'g' is not defined

In [83]:
round(g.rate_limiting_resettime - time.time()) + 5

1182

In [82]:
g.rate_limiting[0]==0

True

In [60]:
# github setup
g = Github(GH_TOKEN)

# point at a repo
repo = g.get_repo("netdata/netdata")

In [61]:
page_len = len(repo.get_stargazers_with_dates().get_page(1))
num_stargazers = repo.get_stargazers().totalCount
num_pages = (num_stargazers // page_len) + 1
print(num_pages)

1334


In [64]:
# list to collect data into
data = []

# loop over pages
for p in range(0,num_pages):
    if p % 100 == 0:
        print(f'page {p} of {num_pages} ({p/num_pages}%)')
    stargazers = repo.get_stargazers_with_dates().get_page(p)
    for stargazer in stargazers:
        row = [
            stargazer.starred_at, stargazer.user.name, stargazer.user.company, 
            stargazer.user.email, stargazer.user.location, stargazer.user.followers,
            stargazer.user.following, stargazer.user.url, stargazer.user.bio,
            stargazer.user.public_repos, stargazer.user.public_gists, stargazer.user.blog,
            stargazer.user.created_at, stargazer.user.updated_at
        ]        
        data.append(row)
        
# make df from data
cols = [
    'starred_at', 'name', 'company', 'email', 'location', 'followers', 
    'following', 'url', 'bio', 'public_repos', 'public_gists', 'blog',
    'created_at', 'updated_at'
]
df = pd.DataFrame(data,columns=cols)

# look at data
print(df.shape)
df.sample(5)

page 0 of 1334 (0.0%)
page 100 of 1334 (0.07496251874062969%)


RateLimitExceededException: 403 {'message': 'API rate limit exceeded for user ID 2178292.', 'documentation_url': 'https://developer.github.com/v3/#rate-limiting'}

In [ ]:
# add email domain 
df['email_domain'] = df['email'].str.split('@').str[-1]

In [ ]:
# add country code via geopy 

# define geo app
geolocator = Nominatim(user_agent="andrewm4894_dev")

# loop over each unique location
user_country_map = dict()
for location in df['location'].unique():
    # default to none
    country_code = None
    # fill it in if possible
    if location != None:
        try: 
            geo_res = do_geocode(location,geolocator)
        except Exception as e: 
            print(e)
            geo_res = None
        if geo_res:
            country_code = geo_res.raw['address']['country_code']
    user_country_map[location] = country_code

# map lookup to col in df
df['country_code'] = df['location'].map(user_country_map)

In [ ]:
# look at data
print(df.shape)
df.sample(5)

In [ ]:
# look at top N stargazers by followers
df.sort_values(by='followers',ascending=False).head(25)

In [ ]:
# look at some plots
for col in ['country_code','company','email_domain']:
    ax = df[col].value_counts(dropna=True).head(20).sort_values().plot(title=col, kind='barh', figsize=(10,6))
    plt.show()